In [ ]:
!pip install pandas matplotlib bokeh seaborn

In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import matplotlib
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, HoverTool
from bokeh.plotting import Figure, output_file, show,output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
dataframe_2015=pd.read_csv("2015.csv")
dataframe_2015['Year']=2015
dataframe_2016=pd.read_csv("2016.csv")
dataframe_2016['Year']=2016

dataframe_2015.set_index('Year',inplace=True)
dataframe_2016.set_index('Year',inplace=True)


In [12]:
data_f=pd.concat([dataframe_2015[1:100],dataframe_2016[1:100]],axis=0)

data_f = data_f.sample(frac=1)


/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [13]:
data_f.head()

,Country,Dystopia Residual,Economy (GDP per Capita),Family,Freedom,Generosity,Happiness Rank,Happiness Score,Health (Life Expectancy),Lower Confidence Interval,Region,Standard Error,Trust (Government Corruption),Upper Confidence Interval
Year,,,,,,,,,,,,,,
2016,Kazakhstan,2.49325,1.22943,0.95544,0.40520,0.15011,54,5.919,0.57386,5.837,Central and Eastern Europe,NaN,0.11132,6.001
2015,Nigeria,2.89319,0.65435,0.90432,0.34334,0.27233,78,5.268,0.16007,NaN,Sub-Saharan Africa,0.04192,0.04030,NaN
2015,Guatemala,2.74255,0.74553,1.04356,0.57733,0.27489,43,6.123,0.64425,NaN,Latin America and Caribbean,0.05224,0.09472,NaN
2015,Macedonia,1.73933,0.91851,1.00232,0.33457,0.22359,93,5.007,0.73545,NaN,Central and Eastern Europe,0.05376,0.05327,NaN
2016,Jamaica,2.35682,0.89333,0.96372,0.43597,0.22245,73,5.510,0.59469,5.315,Latin America and Caribbean,NaN,0.04294,5.705


In [14]:
data_f.columns = data_f.columns.str.replace('\s+', '_') 
data_f.columns = data_f.columns.str.replace('(', '') 
data_f.columns = data_f.columns.str.replace(')', '') 
data_f.columns
data=data_f
data.drop('Dystopia_Residual',axis=1,inplace=True)
data.head(5)

,Country,Economy_GDP_per_Capita,Family,Freedom,Generosity,Happiness_Rank,Happiness_Score,Health_Life_Expectancy,Lower_Confidence_Interval,Region,Standard_Error,Trust_Government_Corruption,Upper_Confidence_Interval
Year,,,,,,,,,,,,,
2016,Kazakhstan,1.22943,0.95544,0.40520,0.15011,54,5.919,0.57386,5.837,Central and Eastern Europe,NaN,0.11132,6.001
2015,Nigeria,0.65435,0.90432,0.34334,0.27233,78,5.268,0.16007,NaN,Sub-Saharan Africa,0.04192,0.04030,NaN
2015,Guatemala,0.74553,1.04356,0.57733,0.27489,43,6.123,0.64425,NaN,Latin America and Caribbean,0.05224,0.09472,NaN
2015,Macedonia,0.91851,1.00232,0.33457,0.22359,93,5.007,0.73545,NaN,Central and Eastern Europe,0.05376,0.05327,NaN
2016,Jamaica,0.89333,0.96372,0.43597,0.22245,73,5.510,0.59469,5.315,Latin America and Caribbean,NaN,0.04294,5.705


In [15]:
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider



slider = Slider(start=0, end=10, value=1, step=.1, title="Slider",bar_color="blue")

show(widgetbox(slider))

In [16]:
source=ColumnDataSource(dict(
x=data.loc[2015].Happiness_Rank,
y=data.loc[2015].Economy_GDP_per_Capita,
country=data.loc[2015].Country,
region=data.loc[2015].Region,
))

plot = Figure(plot_width=900, plot_height=700,
              tools=[HoverTool(tooltips='@country',show_arrow=False)],
              x_axis_label='Happiness Rank',y_axis_label='Economy GDP per Capita')


regions_list = (data.Region.unique()).tolist()

# Import CategoricalColorMapper from bokeh.models 
# and the Spectral6 palette from bokeh.palettes
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral10

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=regions_list, 
                                      palette=Spectral10)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=5, source=source,
            color=dict(field='region',transform=color_mapper), 
            legend='region')
plot.legend.location = 'bottom_right'

In [17]:
from bokeh.io import push_notebook

def update_plot(year):
    
    new_data=dict(
        x=data.loc[year].Happiness_Rank,
        y=data.loc[year].Economy_GDP_per_Capita,
        country=data.loc[year].Country,
        region=data.loc[year].Region,
    )
    source.data=new_data
    push_notebook()
    

In [18]:
from ipywidgets import interact
interact(update_plot,year=(2015,2016,1))
show(plot,notebook_handle=True)
#output_file("jj.html")

interactive(children=(IntSlider(value=2015, description='year', max=2016, min=2015), Output()), _dom_classes=(…